In [ ]:
''' 임시로 엑셀 변환하는 코드 '''
# import os
# import pandas as pd
# # 변환할 CSV 파일의 경로와 이름
# csv_file = "../dataset/본문_형태소분석/2018산림복지_본문_형태소분석(형용사).csv"

# # CSV 파일을 데이터프레임으로 읽기
# dataframe = pd.read_csv(csv_file, encoding='UTF-8')

# # XLSX 파일로 변환
# xlsx_file = os.path.splitext(csv_file)[0] + '.xlsx'
# dataframe.to_excel(xlsx_file, index=False)

In [1]:
import pandas as pd              # pandas 패키지 임포트
import numpy as np               # numpy 패키지 임포트
from konlpy.tag import Okt       # 한글 분석 패키지 konlpy 에서 Okt 형태소 분석기 임포트
from tqdm import tqdm     # for 반복문 진행상황 표시
from collections import Counter
# scikit-learn에서 제공하는 TF-IDF 생성 패키지 임포트
from sklearn.feature_extraction.text import TfidfVectorizer

okt = Okt()   # 코드 단순화를 위해 Okt()를 변수 okt에 할당

In [2]:
# 불용어 목록 만들기
# ※ 분석할 문장에서 조사를 뽑아 유일값만 남기면 됨.

df_josa = pd.read_csv("../dataset/2022/본문_형태소분석/5월/나무문화_5월_형태소분석(조사).csv", encoding='UTF-8')
josa_list = df_josa["조사"].tolist() # 이다, 보다는 지워야 함.
josa_list.remove("이다")
josa_list.remove("보다")
print(len(josa_list))

379


In [3]:
df = pd.read_csv("../dataset/2022/본문_형태소분석/나무문화_본문_5월_형태소분석.txt", encoding='UTF-8')
df

,split_str,org_idx,morphs,pos_tag
0,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,0,"['아테네', '이외', '에도', '그리스', '는', '국토', '전역', '이...","[('아테네', 'Noun'), ('이외', 'Noun'), ('에도', 'Josa..."
1,나는 그중에서도 학창 시절부터 익히 들었던 펠로폰네소스 Peloponnese 라는 ...,0,"['나', '는', '그', '중', '에서도', '학창', '시절', '부터', ...","[('나', 'Noun'), ('는', 'Josa'), ('그', 'Determin..."
2,펠로폰네소스 지역은 그리스 최초의 문화인 미케네 문명이 싹튼 지역이며 병영국가인 스...,0,"['펠로폰네소스', '지역', '은', '그리스', '최초', '의', '문화인',...","[('펠로폰네소스', 'Noun'), ('지역', 'Noun'), ('은', 'Jo..."
3,미케네는 기원전 1500년부터 기원전 1200년까지 소아시아 이집트 키프로스를 대상...,0,"['미케네', '는', '기원전', '1500년', '부터', '기원전', '120...","[('미케네', 'Noun'), ('는', 'Josa'), ('기원전', 'Noun..."
4,미케네인들은 매우 전투적이어서 주변 지역을 비롯해 멀리 소아시아 지역까지 점령했다고...,0,"['미케네', '인들', '은', '매우', '전투', '적', '이어서', '주변...","[('미케네', 'Noun'), ('인들', 'Josa'), ('은', 'Noun'..."
...,...,...,...,...
862581,0 접기 펴기 멋있는 난타공연 난타 공연 너무 멋짐,19230,"['0', '접기', '펴기', '멋있는', '난타', '공연', '난타', '공연...","[('0', 'Number'), ('접기', 'Noun'), ('펴다', 'Verb..."
862582,와 흥이 절로 나고 신났음 ㅋ,19230,"['와', '흥이', '절로', '나고', '신났음', 'ㅋ']","[('오다', 'Verb'), ('흥이', 'Noun'), ('절로', 'Noun'..."
862583,하이라이트 소리지름 주의 LED공연 하이라이트 재생 14 좋아요,19230,"['하이라이트', '소리', '지름', '주의', 'LED', '공연', '하이라이...","[('하이라이트', 'Noun'), ('소리', 'Noun'), ('지름', 'No..."
862584,0 접기 펴기 LED공연 하이라이트 정말 잊을수 없는 공연이었고 못 보고 갔으면 정...,19230,"['0', '접기', '펴기', 'LED', '공연', '하이라이트', '정말', ...","[('0', 'Number'), ('접기', 'Noun'), ('펴다', 'Verb..."


In [4]:
df['joined_morphs'] = df['morphs'].apply(lambda x: ' '.join(x.strip("]['").split("', '")))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs
0,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,0,"['아테네', '이외', '에도', '그리스', '는', '국토', '전역', '이...","[('아테네', 'Noun'), ('이외', 'Noun'), ('에도', 'Josa...",아테네 이외 에도 그리스 는 국토 전역 이 문화유산 이어서 어느 지역 을 가더라도 ...
1,나는 그중에서도 학창 시절부터 익히 들었던 펠로폰네소스 Peloponnese 라는 ...,0,"['나', '는', '그', '중', '에서도', '학창', '시절', '부터', ...","[('나', 'Noun'), ('는', 'Josa'), ('그', 'Determin...",나 는 그 중 에서도 학창 시절 부터 익히 들었던 펠로폰네소스 Peloponnese...
2,펠로폰네소스 지역은 그리스 최초의 문화인 미케네 문명이 싹튼 지역이며 병영국가인 스...,0,"['펠로폰네소스', '지역', '은', '그리스', '최초', '의', '문화인',...","[('펠로폰네소스', 'Noun'), ('지역', 'Noun'), ('은', 'Jo...",펠로폰네소스 지역 은 그리스 최초 의 문화인 미케네 문명 이 싹튼 지역 이며 병영국...
3,미케네는 기원전 1500년부터 기원전 1200년까지 소아시아 이집트 키프로스를 대상...,0,"['미케네', '는', '기원전', '1500년', '부터', '기원전', '120...","[('미케네', 'Noun'), ('는', 'Josa'), ('기원전', 'Noun...",미케네 는 기원전 1500년 부터 기원전 1200년 까지 소아시아 이집트 키프로스 ...
4,미케네인들은 매우 전투적이어서 주변 지역을 비롯해 멀리 소아시아 지역까지 점령했다고...,0,"['미케네', '인들', '은', '매우', '전투', '적', '이어서', '주변...","[('미케네', 'Noun'), ('인들', 'Josa'), ('은', 'Noun'...",미케네 인들 은 매우 전투 적 이어서 주변 지역 을 비롯 해 멀리 소아시아 지역 까...
...,...,...,...,...,...
862581,0 접기 펴기 멋있는 난타공연 난타 공연 너무 멋짐,19230,"['0', '접기', '펴기', '멋있는', '난타', '공연', '난타', '공연...","[('0', 'Number'), ('접기', 'Noun'), ('펴다', 'Verb...",0 접기 펴기 멋있는 난타 공연 난타 공연 너무 멋짐
862582,와 흥이 절로 나고 신났음 ㅋ,19230,"['와', '흥이', '절로', '나고', '신났음', 'ㅋ']","[('오다', 'Verb'), ('흥이', 'Noun'), ('절로', 'Noun'...",와 흥이 절로 나고 신났음 ㅋ
862583,하이라이트 소리지름 주의 LED공연 하이라이트 재생 14 좋아요,19230,"['하이라이트', '소리', '지름', '주의', 'LED', '공연', '하이라이...","[('하이라이트', 'Noun'), ('소리', 'Noun'), ('지름', 'No...",하이라이트 소리 지름 주의 LED 공연 하이라이트 재생 14 좋아요
862584,0 접기 펴기 LED공연 하이라이트 정말 잊을수 없는 공연이었고 못 보고 갔으면 정...,19230,"['0', '접기', '펴기', 'LED', '공연', '하이라이트', '정말', ...","[('0', 'Number'), ('접기', 'Noun'), ('펴다', 'Verb...",0 접기 펴기 LED 공연 하이라이트 정말 잊을수 없는 공연 이었고 못 보고 갔으면...


In [5]:
df['joined_sentences'] = df.groupby('org_idx')['joined_morphs'].transform(lambda x: ' '.join(x))
df

,split_str,org_idx,morphs,pos_tag,joined_morphs,joined_sentences
0,아테네 이외에도 그리스는 국토 전역이 문화유산이어서 어느 지역을 가더라도 고대문화의...,0,"['아테네', '이외', '에도', '그리스', '는', '국토', '전역', '이...","[('아테네', 'Noun'), ('이외', 'Noun'), ('에도', 'Josa...",아테네 이외 에도 그리스 는 국토 전역 이 문화유산 이어서 어느 지역 을 가더라도 ...,아테네 이외 에도 그리스 는 국토 전역 이 문화유산 이어서 어느 지역 을 가더라도 ...
1,나는 그중에서도 학창 시절부터 익히 들었던 펠로폰네소스 Peloponnese 라는 ...,0,"['나', '는', '그', '중', '에서도', '학창', '시절', '부터', ...","[('나', 'Noun'), ('는', 'Josa'), ('그', 'Determin...",나 는 그 중 에서도 학창 시절 부터 익히 들었던 펠로폰네소스 Peloponnese...,아테네 이외 에도 그리스 는 국토 전역 이 문화유산 이어서 어느 지역 을 가더라도 ...
2,펠로폰네소스 지역은 그리스 최초의 문화인 미케네 문명이 싹튼 지역이며 병영국가인 스...,0,"['펠로폰네소스', '지역', '은', '그리스', '최초', '의', '문화인',...","[('펠로폰네소스', 'Noun'), ('지역', 'Noun'), ('은', 'Jo...",펠로폰네소스 지역 은 그리스 최초 의 문화인 미케네 문명 이 싹튼 지역 이며 병영국...,아테네 이외 에도 그리스 는 국토 전역 이 문화유산 이어서 어느 지역 을 가더라도 ...
3,미케네는 기원전 1500년부터 기원전 1200년까지 소아시아 이집트 키프로스를 대상...,0,"['미케네', '는', '기원전', '1500년', '부터', '기원전', '120...","[('미케네', 'Noun'), ('는', 'Josa'), ('기원전', 'Noun...",미케네 는 기원전 1500년 부터 기원전 1200년 까지 소아시아 이집트 키프로스 ...,아테네 이외 에도 그리스 는 국토 전역 이 문화유산 이어서 어느 지역 을 가더라도 ...
4,미케네인들은 매우 전투적이어서 주변 지역을 비롯해 멀리 소아시아 지역까지 점령했다고...,0,"['미케네', '인들', '은', '매우', '전투', '적', '이어서', '주변...","[('미케네', 'Noun'), ('인들', 'Josa'), ('은', 'Noun'...",미케네 인들 은 매우 전투 적 이어서 주변 지역 을 비롯 해 멀리 소아시아 지역 까...,아테네 이외 에도 그리스 는 국토 전역 이 문화유산 이어서 어느 지역 을 가더라도 ...
...,...,...,...,...,...,...
862581,0 접기 펴기 멋있는 난타공연 난타 공연 너무 멋짐,19230,"['0', '접기', '펴기', '멋있는', '난타', '공연', '난타', '공연...","[('0', 'Number'), ('접기', 'Noun'), ('펴다', 'Verb...",0 접기 펴기 멋있는 난타 공연 난타 공연 너무 멋짐,뭉 후의 시선 한국 민속촌 편 벚꽃 이 만 개 할 때 즈음 콧 바람 쐬러 한국 민속...
862582,와 흥이 절로 나고 신났음 ㅋ,19230,"['와', '흥이', '절로', '나고', '신났음', 'ㅋ']","[('오다', 'Verb'), ('흥이', 'Noun'), ('절로', 'Noun'...",와 흥이 절로 나고 신났음 ㅋ,뭉 후의 시선 한국 민속촌 편 벚꽃 이 만 개 할 때 즈음 콧 바람 쐬러 한국 민속...
862583,하이라이트 소리지름 주의 LED공연 하이라이트 재생 14 좋아요,19230,"['하이라이트', '소리', '지름', '주의', 'LED', '공연', '하이라이...","[('하이라이트', 'Noun'), ('소리', 'Noun'), ('지름', 'No...",하이라이트 소리 지름 주의 LED 공연 하이라이트 재생 14 좋아요,뭉 후의 시선 한국 민속촌 편 벚꽃 이 만 개 할 때 즈음 콧 바람 쐬러 한국 민속...
862584,0 접기 펴기 LED공연 하이라이트 정말 잊을수 없는 공연이었고 못 보고 갔으면 정...,19230,"['0', '접기', '펴기', 'LED', '공연', '하이라이트', '정말', ...","[('0', 'Number'), ('접기', 'Noun'), ('펴다', 'Verb...",0 접기 펴기 LED 공연 하이라이트 정말 잊을수 없는 공연 이었고 못 보고 갔으면...,뭉 후의 시선 한국 민속촌 편 벚꽃 이 만 개 할 때 즈음 콧 바람 쐬러 한국 민속...


In [6]:
df_tf = df[['org_idx', 'joined_sentences']].drop_duplicates()
df_tf.reset_index(drop=True, inplace=True)
df_tf

,org_idx,joined_sentences
0,0,아테네 이외 에도 그리스 는 국토 전역 이 문화유산 이어서 어느 지역 을 가더라도 ...
1,1,로얄 엑스 데이지 에이 씨티 그 라운드 ICT 생활 문화센터 2022 05 15 일...
2,2,제 11회 열린 아공 문학상 수상작 각각 미국 프랑스 등 으로 입양 간 어른 이 된...
3,3,4월 부터 열심히 필라테스 하는 중 그룹 레슨 임 에도 기구 나 환경 은 예전 에 ...
4,4,안녕하세요 단단하고 감사하는 최고 구마 입니다 오늘 은 곧 다가오는 6월 광주 문화...
...,...,...
19226,19226,Xosean 출처 Pixabay 어둠 의 단 애 류 인 서 저문다는 것 날 저문다는...
19227,19227,쏜살같이 지나간 4월 을 정리 해보려 한다 내일 5 1 근로자의날 일요일 인거 킹 ...
19228,19228,안녕하세요 좋은 인연 모하 입니다 오늘 은 8 자 씩 공부 하는 천자문 81 번째 ...
19229,19229,안녕하세요 다이어트 매니아 인천 이 찐이야 입니다 워 크 온 이라는 어플 은 걸으면...


In [7]:
# TF-IDF 생성
""" CountVectorizer로 TF-IDF를 만들어서 변환하는 방법도 있지만 """
""" 처음부터 Tfidf를 구해주는 함수(TfidfVectorizer)가 있기 때문에 이를 사용할 것임. """

from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer, TfidfVectorizer

# 데이터 프레임에서 분석할 문서를 가져와서 쓸 경우 꼭 리스트로 변환해서 담아줘야 함.
corpus = df_tf['joined_sentences'].to_list()

In [8]:
# stop_words에는 위에서 조사만 추출해서 만든 불용어 리스트 사용
tfidf = TfidfVectorizer(stop_words = josa_list, token_pattern=r'\w{1,}')
# tfidf_v = TfidfVectorizer(stop_words = hyeong_josa2).fit(corpus)
tdm = tfidf.fit_transform(corpus)

In [9]:
tdm.shape

(19231, 421407)

In [10]:
# 만든 TF-IDF를 데이터 프레임에 담기 위해 우선 numpy.array()로 변환함
tfidf_array = tdm.toarray()
tfidf_array

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.02164712, 0.05849013, 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.01800555, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.06698762, ..., 0.        , 0.        ,
        0.        ],
       [0.07134595, 0.03671914, 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [11]:
# 변환한 array를 다시 pd.Dataframe으로 바꿈
tfidf_DF = pd.DataFrame(tfidf_array)
tfidf_DF

,0,1,2,3,4,5,6,7,8,9,...,421397,421398,421399,421400,421401,421402,421403,421404,421405,421406
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.021647,0.058490,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19226,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19227,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19228,0.000000,0.000000,0.018006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19229,0.000000,0.000000,0.066988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [12]:
# 이 데이터프레임의 컬럼명은 숫자로 되어 있음. 이를 feature_names로 바꿔줘야 함.
featurenames = tfidf.get_feature_names_out()
tfidf_DF.columns = featurenames

In [13]:
tfidf_DF

,0,00,000,0001,0001029084,0001154950,0001213171,0001218,0001524626,0001526232,...,힝들엇읍니,힝또,힝주,힝준,힝헝,힝훙,힝힝,힠힛힛,힣,힣힣
0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.021647,0.058490,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
19226,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19227,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19228,0.000000,0.000000,0.018006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
19229,0.000000,0.000000,0.066988,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [14]:
tfidf_DF.to_csv("../dataset/2022/본문_TF-IDF/5월/나무문화_5월_TF-IDF행렬(원본).txt", index=False, encoding='UTF-8')

In [ ]:
import re
# 데이터프레임에서 한글로만 이루어진 column 필터링을 위한 정규표현식 패턴
pattern = r'^[ㄱ-ㅎㅏ-ㅣ가-힣\s]+$'

# 한글로만 이루어진 column으로 된 데이터프레임 생성
korean_df = tfidf_DF[[column for column in tfidf_DF.columns if re.match(pattern, column)]]

print(korean_df)

In [49]:
# 한글로만 이루어진 TF-IDF 행렬도 txt로 저장해줌.
korean_df.to_csv("../dataset/2022/본문_TF-IDF/5월/나무문화_5월_TF-IDF행렬(원본).txt", index=False, encoding='UTF-8')

In [50]:
# 각 단어에 대한 TF-IDF의 합
word_count = pd.DataFrame({
    '단어': tfidf.get_feature_names_out(),
    'tf-idf': tdm.sum(axis=0).flat
})
word_count

,단어,tf-idf
0,0,51.032181
1,00,50.503403
2,000,19.454386
3,0001,0.049739
4,0001076717,0.034615
...,...,...
134284,힙니,0.062174
134285,힙스터,0.058074
134286,힙합,0.405733
134287,힝,0.086247


In [51]:
# 단어 - TF-IDF 파일로 저장.
word_count.to_csv("../dataset/2022/본문_TF-IDF/1월/나무문화_1월_TF-IDF행렬합.txt", index=False, encoding='UTF-8')